# Environment setups

> Running a Large Language Model using the HuggingFace `Transformers` API.  

In [ ]:
#| default_exp lesson_1.first_run

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Introduction